In [1]:
import socket
import threading

# Function to start a backend server
def start_backend_server(port):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('127.0.0.1', port))
    server_socket.listen(5)
    print(f"Backend server started on port {port}")

    while True:
        client_socket, address = server_socket.accept()
        print(f"Connection received on server {port} from {address}")
        client_socket.sendall(f"Response from server {port}".encode('utf-8'))
        client_socket.close()

# Starting backend servers on ports 9001, 9002, and 9003 in separate threads
for port in [9001, 9002, 9003]:
    thread = threading.Thread(target=start_backend_server, args=(port,))
    thread.daemon = True
    thread.start()


Backend server started on port 9001
Backend server started on port 9002
Backend server started on port 9003


In [3]:
import socket
import threading

# List of backend servers
backend_servers = [('127.0.0.1', 9001), ('127.0.0.1', 9002), ('127.0.0.1', 9003)]

# Round-robin index tracker
round_robin_index = 0

# Function to choose the next backend server using round-robin
def get_next_server():
    global round_robin_index
    server = backend_servers[round_robin_index]
    round_robin_index = (round_robin_index + 1) % len(backend_servers)
    return server

# Function to handle client connection
def handle_client(client_socket):
    server = get_next_server()
    print(f"Routing request to backend server {server}")

    # Connect to the selected backend server
    backend_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    backend_socket.connect(server)

    # Forward the request to the backend server and get the response
    backend_socket.sendall(b"Hello from Load Balancer")
    response = backend_socket.recv(4096)

    # Send the response back to the client
    client_socket.sendall(response)
    client_socket.close()
    backend_socket.close()

# Load balancer function
def start_load_balancer():
    load_balancer_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    load_balancer_socket.bind(('0.0.0.0', 8080))
    load_balancer_socket.listen(5)
    print("Load balancer is running on port 8080...")

    while True:
        client_socket, _ = load_balancer_socket.accept()
        client_handler = threading.Thread(target=handle_client, args=(client_socket,))
        client_handler.start()

# Start the load balancer in a separate thread
load_balancer_thread = threading.Thread(target=start_load_balancer)
load_balancer_thread.daemon = True
load_balancer_thread.start()


Load balancer is running on port 8080...


In [5]:
import socket

# Function to simulate a client sending a request to the load balancer
def simulate_client_request(request_number):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('127.0.0.1', 8080))
    client_socket.sendall(f"Request {request_number}".encode('utf-8')) # What is UTF-8?
    
    response = client_socket.recv(4096)
    print(f"Client received: {response.decode('utf-8')}")
    client_socket.close()

# Send 5 requests to the load balancer
for i in range(1, 6):
    simulate_client_request(i)


Routing request to backend server ('127.0.0.1', 9001)
Connection received on server 9001 from ('127.0.0.1', 60511)
Client received: Response from server 9001
Routing request to backend server ('127.0.0.1', 9002)
Connection received on server 9002 from ('127.0.0.1', 60513)
Client received: Response from server 9002
Routing request to backend server ('127.0.0.1', 9003)
Connection received on server 9003 from ('127.0.0.1', 60515)
Client received: Response from server 9003
Routing request to backend server ('127.0.0.1', 9001)
Connection received on server 9001 from ('127.0.0.1', 60517)
Client received: Response from server 9001
Routing request to backend server ('127.0.0.1', 9002)
Connection received on server 9002 from ('127.0.0.1', 60519)
Client received: Response from server 9002
